<!-- $$V_+(k)=\sqrt{2\pi}\sigma Ae^{-\pi(\sqrt{2\pi}\sigma)^2k^2}$$

$$V_+(x)=Ae^{-\frac{x^2}{2\sigma^2}}$$

$$V_+(k_x,k_y)=(\sqrt{2\pi}\sigma)^2 Ae^{-\pi(\sqrt{2\pi}\sigma)^2({k_x}^2+{k_y}^2)}$$

$$V_+(x,y)=Ae^{-\frac{x^2+y^2}{2\sigma^2}}$$ -->

$$V_+(x,y)=Ae^{-\frac{x^2+y^2}{2\sigma^2}}=Ae^{-\pi\frac{x^2+y^2}{(\sqrt{2\pi}\sigma)^2}}$$
$$V_+(k_x,k_y)=A(\sqrt{2\pi}\sigma)^2e^{-\pi(\sqrt{2\pi}\sigma)^2({k_x}^2+{k_y}^2)}$$

In [205]:
import numpy as np
import tensorflow as tf

import plotly.graph_objects as go

In [652]:
num_quasimomenta = 49
max_momentum_site = 9

max_momentum_site_change = 2 * max_momentum_site
num_momentum_sites = (max_momentum_site_change + 1) ** 2

In [653]:
quasimomentum = np.mgrid[0:1:num_quasimomenta*1j,
                         0:1:num_quasimomenta*1j].reshape(2,-1).T

momentum_site = np.mgrid[-max_momentum_site:max_momentum_site+1:1,
                         -max_momentum_site:max_momentum_site+1:1].reshape(2,-1).T

momentum = 2*momentum_site + quasimomentum[:, np.newaxis]
kinetic = np.linalg.norm(momentum, axis=-1)**2

momentum_site_change = momentum_site[:, np.newaxis] - momentum_site

In [654]:
def render_momentum_site_potential(attractive_strength, repulsive_strength, repulsive_spread):
    attractive_potential = -1/16 * attractive_strength * np.pad(np.array([
        [1, 0, 2, 0, 1],
        [0, 0, 0, 0, 0],
        [2, 0, 4, 0, 2],
        [0, 0, 0, 0, 0],
        [1, 0, 2, 0, 1]
    ]), max_momentum_site_change - 2)

    momentum_site = np.meshgrid(*(np.arange(-max_momentum_site_change, max_momentum_site_change+1),)*2)
    repulsive_potential = repulsive_strength * (np.sqrt(2*np.pi) * repulsive_spread)**2 * np.exp(-np.pi * (np.sqrt(2*np.pi) * repulsive_spread)**2 * (momentum_site[0]**2 + momentum_site[1]**2))

    return attractive_potential + repulsive_potential

In [655]:
def compute_dispersion(momentum_site_potential):
    potential = momentum_site_potential[
        momentum_site_change[:, :, 0] + max_momentum_site_change,
        momentum_site_change[:, :, 1] + max_momentum_site_change,
    ]

    energy = tf.linalg.diag(kinetic) + potential[np.newaxis, ...]

    quarter_dispersion = tf.reshape(tf.linalg.eigvalsh(energy), (num_quasimomenta, num_quasimomenta, num_momentum_sites))
    half_dispersion = np.concatenate((quarter_dispersion, np.flip(quarter_dispersion, axis=0)), axis=0)
    dispersion = np.concatenate((half_dispersion, np.flip(half_dispersion, axis=1)), axis=1)   

    return dispersion

In [656]:
def compute_tunneling(dispersion):
    return np.real(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(dispersion)))) / num_quasimomenta**2

# Analysis

In [657]:
momentum_site_potential = render_momentum_site_potential(0, 0, 0.09)

spatial_potential = np.real(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(momentum_site_potential))))

dispersion = compute_dispersion(momentum_site_potential)

flat_band = dispersion[:, :, 1]
width = np.max(flat_band) - np.min(flat_band)
tunneling = compute_tunneling(flat_band)
tunneling[num_quasimomenta//2, num_quasimomenta//2] = 0
lateral_tunneling = np.abs(tunneling[num_quasimomenta//2, num_quasimomenta//2-1])
score = -width / lateral_tunneling

print(f"Width: {width:.4f}")
print(f"Tunneling: {lateral_tunneling:.6f}")
print(f"Score: {score:.2f}")

Width: 3.0000
Tunneling: 0.000000
Score: -6748931261949064192.00


In [651]:
# figure = go.Figure(data=[go.Surface(z=spatial_potential)])
# figure.update_layout(title="Potential", 
#                      autosize=False, width=750, height=750)
# figure.show()

figure = go.Figure(data=[go.Surface(z=dispersion[:, :, band], showscale=False) for band in range(3)])
figure.update_layout(title="Dispersion",
                     autosize=False, width=750, height=750)
figure.show()

radius = 5
figure = go.Figure(data=[go.Surface(z=tunneling[num_quasimomenta//2-radius:num_quasimomenta//2+radius+1, num_quasimomenta//2-radius:num_quasimomenta//2+radius+1])])
figure.update_layout(title="Tunneling",
                     autosize=False, width=750, height=750)
figure.show()

In [ ]:
# (1, 0) @ (49, 9)
# Width: 2.8751
# Tunneling: 0.210135
# Score: -13.68

# (1, 0) @ (99, 9)
# Width: 2.8751
# Tunneling: 0.205711
# Score: -13.98

# (1, 0) @ (149, 9)
# Width: 2.8751
# Tunneling: 0.204305
# Score: -14.07

In [ ]:
# (50, 0) @ (49, 9)
# Width: 0.1569
# Tunneling: 0.0067
# Score: -23.2786

# (50, 0) @ (99, 9)
# Width: 0.1569
# Tunneling: 0.0066
# Score: -23.8525

# (50, 0) @ (149, 9)
# Width: 0.1569
# Tunneling: 0.0065
# Score: -24.0364

# (50, 0) @ (199, 9)
# Width: 0.1569
# Tunneling: 0.0065
# Score: -24.1269

# (50, 0) @ (249, 9)
# Width: 0.1569
# Tunneling: 0.0065
# Score: -24.1807

# (50, 0) @ (299, 9)
# Width: 0.1569
# Tunneling: 0.0065
# Score: -24.2164